In [2]:
from sklearn.linear_model import Lasso
import pandas as pd
from sklearn.model_selection import KFold, RandomizedSearchCV
import numpy as np
from sklearn.metrics import mean_squared_error
from lags import create_df_with_lags

In [3]:
df = pd.read_csv("../processed_tables/merged_standardized.csv")

## Creación de lags para el precio de la energía

In [4]:
df = create_df_with_lags(df, 3)
df.head()

,precipitacion_amazonas,precipitacion_antioquia,precipitacion_arauca,precipitacion_atlantico,precipitacion_bogota,precipitacion_bolivar,precipitacion_boyaca,precipitacion_caldas,precipitacion_caqueta,precipitacion_casanare,precipitacion_cauca,precipitacion_cesar,precipitacion_choco,precipitacion_cordoba,precipitacion_cundinamarca,precipitacion_guainia,precipitacion_guaviare,precipitacion_huila,precipitacion_la guajira,precipitacion_magdalena,precipitacion_meta,precipitacion_narino,precipitacion_norte de santander,precipitacion_putumayo,precipitacion_quindio,precipitacion_risaralda,precipitacion_san andres providencia,precipitacion_santander,precipitacion_sucre,precipitacion_tolima,precipitacion_valle del cauca,precipitacion_vaupes,precipitacion_vichada,precipitacion_colombia,precipitacion_departamentos_represa,temp_AMAZONAS,temp_ANTIOQUIA,temp_ARAUCA,temp_ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,temp_ATLANTICO,temp_BOGOTA,temp_BOLIVAR,temp_BOYACA,temp_CALDAS,temp_CAQUETA,temp_CASANARE,temp_CAUCA,temp_CESAR,temp_CHOCO,temp_CORDOBA,temp_CUNDINAMARCA,temp_GUAINIA,temp_GUAVIARE,temp_HUILA,temp_LA GUAJIRA,temp_MAGDALENA,temp_META,temp_NARINO,temp_NORTE DE SANTANDER,temp_PUTUMAYO,temp_QUINDIO,temp_RISARALDA,temp_SANTANDER,temp_SUCRE,temp_TOLIMA,temp_VALLE DEL CAUCA,temp_VICHADA,temp_avg_pais,temp_avg_represas,brent_value,TRM,Date,energy_price,energy_price_lag_1,energy_price_lag_2,energy_price_lag_3
3,-0.580600,-0.100687,1.718155,-0.183251,-0.287443,3.802742,0.274778,-0.533335,-0.300951,0.339387,1.246815,-0.336497,0.724237,0.445412,-0.564666,-0.472230,-0.159342,-0.172445,-0.231788,0.155604,0.027750,0.233125,-0.288906,-0.504952,-0.294105,-0.455695,0.416086,0.952586,0.016436,0.575135,0.836212,0.873297,0.532186,0.683349,0.289020,-0.606760,0.531260,3.370191,0.063912,0.438357,1.000139,0.711677,0.906168,0.166109,0.470931,0.532741,0.403566,0.215533,1.293766,0.461229,0.013288,0.316100,0.530335,0.751974,0.969404,0.149209,1.168561,1.612333,0.213532,0.863605,0.110949,0.588202,0.444450,2.107581,1.504325,0.896644,0.057805,1.150745,0.723472,-0.761153,-1.423606,2021-10-04,286.526116,282.065065,256.940174,216.929726
4,0.016831,1.136810,2.981760,0.521690,-0.560886,0.362265,1.586688,0.365171,0.158697,-0.180160,0.441482,0.179796,0.586998,3.276785,-0.427075,-0.565024,0.741289,-0.048627,-0.272739,2.881082,2.346292,0.199601,0.248564,1.188971,-0.444989,-0.325049,-0.266038,0.520671,0.338299,1.267803,1.098443,2.731812,-0.281095,1.284868,1.137304,-0.424941,-0.225111,3.036752,0.407018,0.424930,-0.600440,0.939920,0.262989,-0.694814,-0.016688,-0.130093,0.165056,-0.205928,-0.418129,-0.124853,-0.742700,0.612447,-0.380542,0.252748,0.876107,0.198137,0.430792,0.717358,-0.146140,-0.008211,-1.183151,-0.106008,-0.087541,-0.365573,0.219063,-0.446079,-0.701899,0.112145,-0.197147,-0.614059,-1.411032,2021-10-05,278.157325,286.526116,282.065065,256.940174
5,-0.142496,3.004638,4.689849,-0.306373,-0.567283,0.898967,2.674880,0.219436,2.305734,0.758029,0.654533,-0.329659,0.932841,2.563767,-0.090468,-0.085590,1.498694,0.284344,-0.363214,-0.153043,1.356370,-0.213769,-0.214504,-0.167175,-0.418644,-0.536092,1.152583,-0.029673,1.671145,1.498216,1.694128,-1.166024,-0.409508,1.614504,1.733979,0.213640,0.010703,2.937955,0.559510,-0.488095,-0.129682,0.073942,-0.483831,-0.845177,-0.102667,-0.504474,-0.165863,-0.214401,0.235510,-1.711884,-1.199478,-0.318928,-1.784190,-0.025725,0.390964,-0.451912,0.133205,1.131521,-0.369521,-1.403118,-0.679179,-0.097308,-0.481705,-1.473066,0.764819,0.080976,1.235347,-0.157838,-0.525688,-0.624360,-1.383612,2021-10-06,261.474788,278.157325,286.526116,282.065065
6,0.132014,0.622153,1.953096,-0.256603,-0.572080,-0.005022,0.895201,-0.454157,-0.524727,-0.042188,0.464208,-0.379237,0.803226,-0.269907,-0.557164,4.267981,-0.494348,0.014956,-0.335913,0.006676,0.720542,-0.386837,0.150842,-0.015932,-0.454569,-0.408796,-0.340182,1.731427,-0.028955,-0.386586,0.454196,-0.606787,0.740857,0.466008,0.329439,-0.021393,0.363389,2.913256,0.788248,0.545772,0.105698

## Partición de la data

Debido a que la naturaleza de la data es de carácter temporal, se realizará una partición de la data en 3 partes: entrenamiento y prueba. La data de test será los últimos 3 meses de la data, es decir desde julio 2023 hasta septiembre 2023.


In [5]:
fecha_corte = "2023-07-01"
X_train = df[df["Date"] < fecha_corte].drop(["Date", "energy_price"], axis=1)
X_test = df[df["Date"] >= fecha_corte].drop(["Date", "energy_price"], axis=1)
y_train = df[df["Date"] < fecha_corte]["energy_price"]
y_test = df[df["Date"] >= fecha_corte]["energy_price"]

## Regresión lineal

Esta vez utilizaremos `sklearn` para hacer la regresión Lasso y poder ver qué coeficientes son cero.


In [6]:
lasso = Lasso()

# Define the parameter grid to search
param_grid = {'alpha': np.logspace(-4, 4, 20)}

# Setup Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Setup Randomized Grid Search
lasso_cv = RandomizedSearchCV(lasso, param_grid, cv=kfold, random_state=42)

# Fit the model
lasso_cv.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters found: ", lasso_cv.best_params_)
print("Best cross-validation score: ", lasso_cv.best_score_)

/home/david/.local/share/virtualenvs/Proyecto-Integrador-Semestre1-ERzOjjfT/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.326e+05, tolerance: 1.280e+03
  model = cd_fast.enet_coordinate_descent(
/home/david/.local/share/virtualenvs/Proyecto-Integrador-Semestre1-ERzOjjfT/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.834e+05, tolerance: 1.412e+03
  model = cd_fast.enet_coordinate_descent(
/home/david/.local/share/virtualenvs/Proyecto-Integrador-Semestre1-ERzOjjfT/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning

Best parameters found:  {'alpha': 4.281332398719396}
Best cross-validation score:  0.9406517684146578


In [7]:
y_pred = lasso_cv.predict(X_test)

In [8]:
mse_test = mean_squared_error(y_test, y_pred)

print("Mean Squared Error on Test Data:", mse_test)

Mean Squared Error on Test Data: 4491.264876612977


In [9]:
coefficients = lasso_cv.best_estimator_.coef_

In [10]:
feature_names = X_train.columns

# Filter the coefficients and corresponding feature names
non_zero_coefficients = coefficients[coefficients != 0]
non_zero_features = feature_names[coefficients != 0]

# Print the non-zero coefficients and their corresponding feature names
for feature, coef in zip(non_zero_features, non_zero_coefficients):
    print(f"{feature}: {coef}")

precipitacion_meta: -0.6033561644760532
precipitacion_tolima: -0.34569551052102393
temp_BOYACA: 0.7436602031153009
temp_QUINDIO: 0.5733149923544618
temp_VALLE DEL CAUCA: 1.149911961572405
energy_price_lag_1: 1.0746413836470614
energy_price_lag_2: 0.06520560726103745
energy_price_lag_3: -0.18143605444202718
